# ДЗ 3. Группировка данных и оконные функции

Задачи:

1. Создать таблицы со следующими структурами и загрузить данные из csv-файлов;

2. Выполнить следующие запросы:

* Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества;

* Найти сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности;

* Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT;

* Найти по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, отсортировав результат по убыванию суммы транзакций и количества клиентов. Выполните двумя способами: используя только group by и используя только оконные функции. Сравните результат;

* Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). Напишите отдельные запросы для минимальной и максимальной суммы;

* Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций;

* Вывести имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях).

# Решение

### Посмотрим на исходный датасет

In [1]:
import pandas as pd

Customer:

In [2]:
customer_df = pd.read_csv('./data/customer.csv', sep=';')
customer_df.head()

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,Eli,Bockman,Male,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,3,Arlin,Dearle,Male,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,0 Holy Cross Court,4211,QLD,Australia,9
3,4,Talbot,NaN,Male,1961-10-03,NaN,IT,Mass Customer,N,No,17979 Del Mar Point,2448,New South Wales,Australia,4
4,5,Sheila-kathryn,Calton,Female,1977-05-13,Senior Editor,NaN,Affluent Customer,N,Yes,9 Oakridge Court,3216,VIC,Australia,9


In [3]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   customer_id            4000 non-null   int64 
 1   first_name             4000 non-null   object
 2   last_name              3875 non-null   object
 3   gender                 4000 non-null   object
 4   DOB                    3913 non-null   object
 5   job_title              3494 non-null   object
 6   job_industry_category  3344 non-null   object
 7   wealth_segment         4000 non-null   object
 8   deceased_indicator     4000 non-null   object
 9   owns_car               4000 non-null   object
 10  address                4000 non-null   object
 11  postcode               4000 non-null   int64 
 12  state                  4000 non-null   object
 13  country                4000 non-null   object
 14  property_valuation     4000 non-null   int64 
dtypes: int64(3), object(1

Transaction:

In [4]:
transaction_df = pd.read_csv('./data/transaction.csv', sep=';')
transaction_df.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,1,2,2950,25.02.2017,False,Approved,Solex,Standard,medium,medium,"71,49","53,62"
1,2,3,3120,21.05.2017,True,Approved,Trek Bicycles,Standard,medium,large,"2091,47","388,92"
2,3,37,402,16.10.2017,False,Approved,OHM Cycles,Standard,low,medium,"1793,43","248,82"
3,4,88,3135,31.08.2017,False,Approved,Norco Bicycles,Standard,medium,medium,"1198,46","381,10"
4,5,78,787,01.10.2017,True,Approved,Giant Bicycles,Standard,medium,large,"1765,3","709,48"


In [5]:
transaction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   transaction_id    20000 non-null  int64 
 1   product_id        20000 non-null  int64 
 2   customer_id       20000 non-null  int64 
 3   transaction_date  20000 non-null  object
 4   online_order      19640 non-null  object
 5   order_status      20000 non-null  object
 6   brand             19803 non-null  object
 7   product_line      19803 non-null  object
 8   product_class     19803 non-null  object
 9   product_size      19803 non-null  object
 10  list_price        20000 non-null  object
 11  standard_cost     19803 non-null  object
dtypes: int64(3), object(9)
memory usage: 1.8+ MB


Вывод: на первый взгляд в исходных таблицах аномалий кроме пропусков не обнаружено

### Создаем таблицы в DBeaver

![createtable](https://thumb.cloud.mail.ru/weblink/thumb/xw1/Jekx/7J88b15xj)

### Загружаем данные в таблицы

Customer:

![add_data_customer](https://thumb.cloud.mail.ru/weblink/thumb/xw1/xPen/UWCCjvwBG)

Transaction:

При загрузке данных в таблицу transaction произошла ошибка:

![add_data_customer](https://thumb.cloud.mail.ru/weblink/thumb/xw1/h1s2/apYPVCFF7)

Удалим строки c customer_id = 5034 и перезапишем таблицу transaction:

In [6]:
transaction_df = transaction_df.query('customer_id != 5034')

In [7]:
transaction_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19997 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   transaction_id    19997 non-null  int64 
 1   product_id        19997 non-null  int64 
 2   customer_id       19997 non-null  int64 
 3   transaction_date  19997 non-null  object
 4   online_order      19637 non-null  object
 5   order_status      19997 non-null  object
 6   brand             19800 non-null  object
 7   product_line      19800 non-null  object
 8   product_class     19800 non-null  object
 9   product_size      19800 non-null  object
 10  list_price        19997 non-null  object
 11  standard_cost     19800 non-null  object
dtypes: int64(3), object(9)
memory usage: 2.0+ MB


In [8]:
transaction_df.to_csv('./data/transaction_5034.csv', index=False)

Загружаем данные повторно:

![add_data_transaction](https://thumb.cloud.mail.ru/weblink/thumb/xw1/QZ9X/H8wRn2Nz6)

### Выполняем запросы

1. Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества.

![1](https://thumb.cloud.mail.ru/weblink/thumb/xw1/nPJ6/P9fjLhVcz)

2. Найти сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности.

![2](https://thumb.cloud.mail.ru/weblink/thumb/xw1/E8kv/YqWUQFYFz)

3. Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT.

![3](https://thumb.cloud.mail.ru/weblink/thumb/xw1/FoTp/KZL2KPHTp)

4. Найти по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, отсортировав результат по убыванию суммы транзакций и количества клиентов. Выполните двумя способами: используя только group by и используя только оконные функции. Сравните результат.

Group by:

![4.1](https://thumb.cloud.mail.ru/weblink/thumb/xw1/RMsL/Ddi2qMicj)

Оконные функции:

![4.2](https://thumb.cloud.mail.ru/weblink/thumb/xw1/gZgM/97Boj8Uvp)

Сравнение: при выполнении запрпоса с помощью группировки (group by) мы автоматически получаем таблицу с уникальными значениями customer_id, а при выполнении запроса с окнными функциями чтобы исключитиь дубликаты по полю customer_id необходимо использовать distinct, иначе выводится вся таблица.

5. Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). Напишите отдельные запросы для минимальной и максимальной суммы.

Клиент с минимальной суммой транзакций:

![5.1](https://thumb.cloud.mail.ru/weblink/thumb/xw1/vVv1/f8s8kmEqt)

Клиент с максимальной суммой транзакций:

![5.2](https://thumb.cloud.mail.ru/weblink/thumb/xw1/6PTY/KnskPhLwk)

6. Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций.

![6](https://thumb.cloud.mail.ru/weblink/thumb/xw1/F3rV/UK8CBb762)

7. Вывести имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях).

![7](https://thumb.cloud.mail.ru/weblink/thumb/xw1/k57X/myCzdZfaX)